# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir

import os

from azureml.widgets import RunDetails
from azureml.core import Workspace, Experiment, Dataset, Environment, ScriptRunConfig
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

Found existing installation: ipywidgets 7.6.5
Uninstalling ipywidgets-7.6.5:
  Successfully uninstalled ipywidgets-7.6.5
Found existing installation: azureml-widgets 1.60.0
Uninstalling azureml-widgets-1.60.0:
  Successfully uninstalled azureml-widgets-1.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 155.5 MB/s eta 0:00:00 0:00:01m


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'myhyperdriveexp'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling =  RandomParameterSampling(
    {
        '--C': uniform(0.01, 100),
        '--max_iter': choice(50, 100, 200, 300)
    }
)

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
estimator = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    environment=sklearn_env,
    compute_target='mycompute'
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=30
)

In [4]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

2025-06-22 21:26:20.425544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750627580.452721   32039 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750627580.460571   32039 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750627580.480917   32039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750627580.480944   32039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750627580.480946   32039 computation_placer.cc:177] computation placer alr

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_00c76c52-899c-432e-9c54-9e7d9aae225d
Web View: https://ml.azure.com/runs/HD_00c76c52-899c-432e-9c54-9e7d9aae225d?wsid=/subscriptions/18232d0b-4c73-4051-8694-16fb46b5a555/resourcegroups/myfreeresource/workspaces/MyFreeWorkspace&tid=210e9353-d35d-46b6-a109-61879c12384a

Streaming azureml-logs/hyperdrive.txt

[2025-06-22T21:26:17.3246477Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-06-22T21:26:17.6005297Z][SCHEDULER][INFO]Scheduling job, id='HD_00c76c52-899c-432e-9c54-9e7d9aae225d_0' 
[2025-06-22T21:26:17.6458319Z][SCHEDULER][INFO]Scheduling job, id='HD_00c76c52-899c-432e-9c54-9e7d9aae225d_1' 
[2025-06-22T21:26:17.7804373Z][SCHEDULER][INFO]Scheduling job, id='HD_00c76c52-899c-432e-9c54-9e7d9aae225d_2' 
[2025-06-22T21:26:17.7818116Z][SCHEDULER][INFO]Scheduling job, id='HD_00c76c52-899c-432e-9c54-9e7d9aae225d_3' 
[2025-06-22T21:26:17.8837104Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_00c76c52-899c-432e-9c54-9e7d9aae225d_0' 
[2025-06-22T21:26:17.9244114Z][

{'runId': 'HD_00c76c52-899c-432e-9c54-9e7d9aae225d',
 'target': 'mycompute',
 'status': 'Completed',
 'startTimeUtc': '2025-06-22T21:26:15.846119Z',
 'endTimeUtc': '2025-06-22T21:57:13.800372Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5aa3fda5-d66d-47c7-bdab-5ca13cbda390',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_00c76c52-899c-432e-9c54-9e7d9aae225d_21',
  'score': '0.7931769722814499',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_00c76c52-899c-432e-9c54-9e7d9aae225d_21'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-tr

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
import joblib
from azureml.core.model import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Parameters:',parameter_values)

Best Run Id:  HD_00c76c52-899c-432e-9c54-9e7d9aae225d_21

 Accuracy: 0.7931769722814499

 Parameters: ['--C', '85.19735118470717', '--max_iter', '100']


In [7]:
#TODO: Save the best model
model_file_name = 'hyperdriveBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

print(f"Registering model '{model_file_name}' from run {best_run.id}...")
model = Model.register(workspace=ws,
                       model_path=local_model_path, # Path to the downloaded model file
                       model_name="best_hyperdrive_model", # Name for your registered model
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('Accuracy', 'N/A')},
                       description="Logistic Regression model tuned with HyperDrive")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

Model downloaded successfully.
Registering model 'hyperdriveBestModel.pkl' from run HD_00c76c52-899c-432e-9c54-9e7d9aae225d_21...
Registering model best_hyperdrive_model
Model registered successfully with name: best_hyperdrive_model, version: 1


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

